In [3]:
%pip install azure-ai-projects 
# azure-ai-agentserver-core azure-ai-agentserver-agentframework

Note: you may need to restart the kernel to use updated packages.


In [ ]:
! az acr login -n acr4hostedagent --expose-token
! az acr build -r acr4hostedagent -t agents/web-search-agent ./web-search-agent

In [25]:
foundry_project_endpoint = ! terraform output -raw foundry_project_endpoint
foundry_project_endpoint = foundry_project_endpoint.n
print("Foundry Project Endpoint: ", foundry_project_endpoint)

foundry_api_key = ! terraform output -raw foundry_api_key
foundry_api_key = foundry_api_key.n
print("Foundry API Key: ", foundry_api_key)

llm_model_deployment_name = ! terraform output -raw llm_model_deployment_name
llm_model_deployment_name = llm_model_deployment_name.n
print("LLM Model Deployment Name: ", llm_model_deployment_name)

agent_container = "acr4hostedagent.azurecr.io/agents/web-search-agent:latest"

foundry_name = ! terraform output -raw foundry_name
foundry_name = foundry_name.n
print("Foundry Name: ", foundry_name)

foundry_project_name = ! terraform output -raw foundry_project_name
foundry_project_name = foundry_project_name.n
print("Foundry Project Name: ", foundry_project_name)

hosted_agent_name = "agent-web-search"
hosted_agent_version = "v1"

import os
os.environ["NO_PROXY"] = "*"

Foundry Project Endpoint:  https://foundry-270us-test.services.ai.azure.com/api/projects/project-270us-test
Foundry API Key:  7lBGRTza0ckt3sZzkXOwNgNmN9apAciJKgXsLswzYRjbC7PrvVQ9JQQJ99CAACHrzpqXJ3w3AAAAACOG8G4L
LLM Model Deployment Name:  gpt-4o-mini
Foundry Name:  foundry-270us-test
Foundry Project Name:  project-270us-test


In [26]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ImageBasedHostedAgentDefinition, ProtocolVersionRecord, AgentProtocol
from azure.identity import DefaultAzureCredential

# Initialize the client
client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential()
)

# Create the agent from a container image
agent = client.agents.create_version(
    agent_name=hosted_agent_name,
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")],
        cpu="1",
        memory="2Gi",
        image=agent_container,
        environment_variables={
            "AZURE_AI_PROJECT_ENDPOINT": foundry_project_endpoint,
            "API_KEY": foundry_api_key,
            "MODEL_NAME": llm_model_deployment_name,
            "CUSTOM_SETTING": "value"
        }
    )
)

## Start an agent deployment

After you create your hosted agent version, you can start the deployment by using the az CLI extension to make it available for requests. You can also start a hosted agent that's stopped.

In [21]:
! az cognitiveservices agent start --account-name {foundry_name} --project-name {foundry_project_name} --name {hosted_agent_name} --agent-version {hosted_agent_version}

D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foundry-270us-test.services.ai.azure.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
ERROR: (invalid_parameters) Invalid format.
Code: invalid_parameters
Message: Invalid format.


If starting the hosted agent using the above command line fails, you can start it using the Foundry portal as shown below.

![Starting the agent on the Foundry portal](./images/start-agent.png)